In [1]:
# Import all needed libraries

# ! pip install google-cloud-bigquery
from google.cloud import bigquery
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
#from config import db_password

In [2]:
# GCP Authentication
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="client_secret.json"

In [3]:
client = bigquery.Client()

In [4]:
# Exploring raw dataset
hn_dataset_ref = client.dataset('stackoverflow', project='bigquery-public-data')
hn_dataset_ref

DatasetReference('bigquery-public-data', 'stackoverflow')

In [5]:
type(hn_dataset_ref)

google.cloud.bigquery.dataset.DatasetReference

In [6]:
hn_dset = client.get_dataset(hn_dataset_ref)
hn_dset

Dataset(DatasetReference('bigquery-public-data', 'stackoverflow'))

In [7]:
type(hn_dset)

google.cloud.bigquery.dataset.Dataset

In [8]:
# Determine available data tables
[x.table_id for x in client.list_tables(hn_dset)]

['badges',
 'comments',
 'post_history',
 'post_links',
 'posts_answers',
 'posts_moderator_nomination',
 'posts_orphaned_tag_wiki',
 'posts_privilege_wiki',
 'posts_questions',
 'posts_tag_wiki',
 'posts_tag_wiki_excerpt',
 'posts_wiki_placeholder',
 'stackoverflow_posts',
 'tags',
 'users',
 'votes']

In [9]:
# Exploring data in posts_questions
hn_full = client.get_table(table='bigquery-public-data.stackoverflow.posts_questions')
query_job = client.query(
        """
        SELECT
        *
        FROM `bigquery-public-data.stackoverflow.posts_questions`
        WHERE accepted_answer_id is not null AND creation_date > '2021-01-01'
        ORDER BY accepted_answer_id
        LIMIT 10"""
    )

results = query_job.result() 
results_list = [row for row in results]
cols = [field.name for field in results.schema]
posts_questions = pd.DataFrame.from_records(results_list, columns=cols)
posts_questions

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,65526420,How to check if values in individiual rows of ...,<p>Suppose I have the following data.table:</p...,65526457,2,2,None,2021-01-01 00:05:46.310000+00:00,2.0,2021-01-01 02:15:42.223000+00:00,NaT,None,NaN,None,6335637,None,1,2,r|data.table|rowwise,62
1,65526423,d3.js : Generating axis ticks for ordinal values,<p>I want to use ordinal scale in x-axis with ...,65526533,1,0,None,2021-01-01 00:06:09.007000+00:00,NaN,2021-01-01 16:25:58.447000+00:00,NaT,None,NaN,None,9574155,None,1,2,javascript|d3.js,38
2,65526490,Is there a C macro that replaces varied length...,<p>I want to be able to:</p>\n<pre><code>#defi...,65526541,1,1,None,2021-01-01 00:20:09.553000+00:00,0.0,2021-01-01 00:37:24.277000+00:00,NaT,None,NaN,None,12743240,None,1,2,c|macros,35
3,65526419,How can I construct my objects allocated throu...,<p>C++20 removed the <code>construct()</code> ...,65526554,2,0,None,2021-01-01 00:05:43.627000+00:00,NaN,2021-01-01 18:19:46.870000+00:00,2021-01-01 05:23:28.560000+00:00,None,65863.0,None,14694500,None,1,3,c++|std|c++20|allocator,178
4,65526523,Navigate from parent to child in react where e...,<p>In a React app with a parent and child elem...,65526577,1,1,None,2021-01-01 00:30:31.933000+00:00,NaN,2021-01-01 00:46:22.813000+00:00,2021-01-01 00:35:53.623000+00:00,None,8690857.0,None,3002584,None,1,1,reactjs|react-router|react-router-dom,33
5,65526451,How do I define a field in a subclass by stric...,<p>I'm trying to implement the UML diagram bel...,65526589,1,0,None,2021-01-01 00:11:17.787000+00:00,NaN,2021-01-01 10:22:51.790000+00:00,2021-01-01 10:22:51.790000+00:00,None,2458991.0,None,14634129,None,1,1,java|inheritance|uml,50
6,65526591,How to install Python 3.5.x on Ubuntu 18.04 LT...,<p>Simple question: How to install Python 3.5....,65526611,2,1,None,2021-01-01 00:49:45.320000+00:00,NaN,2021-01-01 09:48:41.703000+00:00,NaT,None,NaN,None,6305105,None,1,0,python|ubuntu,73
7,65526474,GDB: Displaying incorrect values in struct,<p>I'm trying to implement the <code>malloc</c...,65526637,1,0,None,2021-01-01 00:16:10.673000+00:00,NaN,2021-01-01 01:02:22.447000+00:00,2021-01-01 00:39:32.250000+00:00,None,1233251.0,None,11035194,None,1,1,struct|gdb|memory-address,32
8,65526590,Vue2 + laravel6 - Component implementation,<p>I just started using Vue2 with Laravel6 and...,65526639,1,0,None,2021-01-01 00:49:32.030000+00:00,NaN,2021-01-01 01:09:45.250000+00:00,2021-01-01 01:07:22.133000+00:00,None,11960598.0,None,14605909,None,1,0,laravel|vue.js|vuejs2|laravel-6,37
9,65526522,Mobile menu css,<p>What's the best way to\nachieve going from ...,65526667,2,0,None,2021-01-01 00:30:28.633000+00:00,NaN,2021-01-01 01:46:35.280000+00:00,NaT,None,NaN,None,6787542,None,1,0,css,38


In [51]:
# Extract data for May 2021 for posts with accepted ansers
query_job = client.query(
        """
        SELECT
        id, title, body, accepted_answer_id, creation_date, favorite_count, score, tags, view_count
        FROM `bigquery-public-data.stackoverflow.posts_questions`
        WHERE accepted_answer_id is not null AND creation_date > '2021-05-01'
        ORDER BY accepted_answer_id"""
    )

results = query_job.result() 
results_list = [row for row in results]
cols = ['q_id', 'q_title', 'q_body', 'accepted_answer_id', 'q_creation_date', 'q_favorite_count', 'q_score', 'q_tags', 'q_view_count']
posts_questions_df = pd.DataFrame.from_records(results_list, columns=cols)
posts_questions_df.sample(n=15)

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_favorite_count,q_score,q_tags,q_view_count
44271,67725227,Nodejs change time at date type after getting ...,<p>I wanted to change time at date type which ...,67725612,2021-05-27 15:22:08.413000+00:00,NaN,0,node.js|mongodb,29
4949,67388008,Why doesn't calling a substring using the leng...,"<p>When was testing some code, I noticed that ...",67388274,2021-05-04 15:47:29.327000+00:00,NaN,-1,java,32
21686,67527137,Discord bot traceback key error: '736458231848...,<p>I am trying to make a Discord bot with a yo...,67529768,2021-05-13 22:52:52.057000+00:00,1.0,-3,python|discord.py|bots,34
40813,67692282,fgets() doesn't ask for input the second time ...,<p>I was writing code which called 2 different...,67692932,2021-05-25 16:44:03.697000+00:00,NaN,0,c|fgets,72
3443,67374122,Python applying a method to elements pairs fro...,<p>I have two lists containing string - The si...,67374254,2021-05-03 18:36:53.527000+00:00,NaN,0,python|list|pairwise,33
43014,67713004,How to repeat values for different variables i...,<p>I want to repeat the information of these v...,67713038,2021-05-26 21:35:24.770000+00:00,NaN,1,r|database|na,20
18985,67507568,Creating column to store multiplied values fro...,<p>I'm new to the SQL and I have a problem wit...,67507589,2021-05-12 16:29:51.613000+00:00,NaN,0,sql|oracle|ddl,17
28353,67584186,Unable to add new dependency in react application,<p>I have a react application. I added a new d...,67585092,2021-05-18 10:12:20.787000+00:00,1.0,1,javascript|node.js|reactjs|node-modules,50
362,67345398,Pass data from Controller to JS in Odoo,<p>How can you pass data from a controller to ...,67346076,2021-05-01 10:51:05.930000+00:00,NaN,0,javascript|python|odoo|odoo-14,30
31250,67608786,Numpy condition on a vector rather than an ele...,<p>I have a numpy array that represents an ima...,67609823,2021-05-19 18:18:02.657000+00:00,NaN,0,python|numpy|numpy-ndarray,62


In [52]:
# Determine data types for posts_questions_df
posts_questions_df.dtypes

q_id                                int64
q_title                            object
q_body                             object
accepted_answer_id                  int64
q_creation_date       datetime64[ns, UTC]
q_favorite_count                  float64
q_score                             int64
q_tags                             object
q_view_count                        int64
dtype: object

In [53]:
# How many missing values in the DF by column
[[column,posts_questions_df[column].isnull().sum()] for column in posts_questions_df.columns]

[['q_id', 0],
 ['q_title', 0],
 ['q_body', 0],
 ['accepted_answer_id', 0],
 ['q_creation_date', 0],
 ['q_favorite_count', 43400],
 ['q_score', 0],
 ['q_tags', 0],
 ['q_view_count', 0]]

In [54]:
len(posts_questions_df)

47057

In [56]:
# Drop q_favorite_count column because too many null values
posts_questions_df = posts_questions_df.drop(columns='q_favorite_count')
posts_questions_df.columns

KeyError: "['q_favorite_count'] not found in axis"

In [57]:
# How many missing values in the DF by column
[[column,posts_questions_df[column].isnull().sum()] for column in posts_questions_df.columns]

[['q_id', 0],
 ['q_title', 0],
 ['q_body', 0],
 ['accepted_answer_id', 0],
 ['q_creation_date', 0],
 ['q_score', 0],
 ['q_tags', 0],
 ['q_view_count', 0]]

In [58]:
# Additional method to verify number of rows with zero response time (what's the possible explanation?)
q_score_explore_raw = posts_questions_df.groupby('q_score').count()
q_score_explore_raw

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count
q_score,,,,,,,
-9,1,1,1,1,1,1,1
-8,2,2,2,2,2,2,2
-7,2,2,2,2,2,2,2
-6,13,13,13,13,13,13,13
-5,37,37,37,37,37,37,37
-4,152,152,152,152,152,152,152
-3,440,440,440,440,440,440,440
-2,1111,1111,1111,1111,1111,1111,1111
-1,3218,3218,3218,3218,3218,3218,3218


In [59]:
# Bin score for more meaningful analysis
# Establish the bins.
q_score_bins = [-10, -1, 0, 10, 25, 50, 101]
q_score_group_names = ["Negative Score (<0)", "Zero Score (0)", "Low Score (0-10)", "Medium Score (10 - 25)", "High Score (25-50)", "Popular Score (50-101)"]

# Categorize score based on the bins.
posts_questions_df['q_score_tier'] = pd.cut(posts_questions_df['q_score'], q_score_bins, labels=q_score_group_names)

In [61]:
# explore scores
q_score_explore = posts_questions_df.groupby('q_score_tier').count()
q_score_explore

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_score,q_tags,q_view_count
q_score_tier,,,,,,,,
Negative Score (<0),4976,4976,4976,4976,4976,4976,4976,4976
Zero Score (0),26580,26580,26580,26580,26580,26580,26580,26580
Low Score (0-10),15469,15469,15469,15469,15469,15469,15469,15469
Medium Score (10 - 25),21,21,21,21,21,21,21,21
High Score (25-50),7,7,7,7,7,7,7,7
Popular Score (50-101),4,4,4,4,4,4,4,4


In [63]:
# Bin score for more meaningful analysis (broader bin)

# Establish the bins.
q_score_bins = [-10, -1, 0, 101]
q_score_group_names = ["Negative Score (<0)", "Zero Score (0)", "Positive Score (>0)"]
# Categorize score based on the bins.
posts_questions_df['q_score_tier'] = pd.cut(posts_questions_df['q_score'], q_score_bins, labels=q_score_group_names)
# Bin scores
q_score_explore = posts_questions_df.groupby('q_score_tier').count()
q_score_explore

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_score,q_tags,q_view_count
q_score_tier,,,,,,,,
Negative Score (<0),4976,4976,4976,4976,4976,4976,4976,4976
Zero Score (0),26580,26580,26580,26580,26580,26580,26580,26580
Positive Score (>0),15501,15501,15501,15501,15501,15501,15501,15501


In [64]:
# Verify whether there are redundant columns for score_tier
posts_questions_df.columns

Index(['q_id', 'q_title', 'q_body', 'accepted_answer_id', 'q_creation_date',
       'q_score', 'q_tags', 'q_view_count', 'q_score_tier'],
      dtype='object')

In [66]:
# Verify data in q_score_tier column
posts_questions_df.sample(n=10)

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_score,q_tags,q_view_count,q_score_tier
21763,67497458,Get files from a specific directory/folder in ...,<p>I need to list all files from a specific di...,67530352,2021-05-12 04:44:06.820000+00:00,1,.net|azure|azure-storage-blobs|azure-blob-storage,58,Positive Score (>0)
45337,67736397,redeclare const variable in javascript,<p>One of the key feature of <code>const</code...,67736506,2021-05-28 09:34:51.690000+00:00,0,javascript|constants,40,Zero Score (0)
33904,67627228,XSLT: Override fn:generate-id function to crea...,<p>We depend on an external XSLT library that ...,67632842,2021-05-20 20:03:51.470000+00:00,0,xslt|saxon,28,Zero Score (0)
853,67345763,How to count number of tweets via Twitter API,<p>Is it possible to send a request to Twitter...,67351171,2021-05-01 11:29:19.703000+00:00,1,twitter|tweepy|twitterapi-python,24,Positive Score (>0)
4682,67385771,uploading and retrieving image from firebase,<p>I am using firebase to upload a photo of a ...,67385929,2021-05-04 13:29:23.530000+00:00,0,javascript|firebase|react-native|firebase-storage,19,Zero Score (0)
41799,67701539,Calling Grandfather class method results in Re...,<p>Im trying to call a grandfather method and ...,67701630,2021-05-26 08:47:52.127000+00:00,1,python|recursion|inheritance,31,Positive Score (>0)
14632,67470837,Why is my Dependency Property Binding is not w...,<p>I am trying to make a customized UserContro...,67470898,2021-05-10 12:56:47.867000+00:00,0,c#|wpf|mvvm|binding|dependency-properties,26,Zero Score (0)
15245,67475766,Data class for API response structure?,<p>I am using wikipedia API whose response str...,67475955,2021-05-10 18:24:43.063000+00:00,-1,android|json|kotlin|retrofit|data-class,24,Negative Score (<0)
36294,67626054,Implementing Weighted Mean Absolute/Square dif...,<p>I am trying to implement some sort of corre...,67653508,2021-05-20 18:24:24.317000+00:00,0,python|numpy|opencv|image-processing|scipy,76,Zero Score (0)
38725,67667622,Cannot get all the document inside sub collect...,"<p>I have a collection name items, with docume...",67674584,2021-05-24 06:37:17.363000+00:00,0,javascript|firebase|google-cloud-firestore,23,Zero Score (0)


In [67]:
# Drop score column because data transformed to the score_tier column
posts_questions_df = posts_questions_df.drop(columns='q_score')
posts_questions_df.columns

Index(['q_id', 'q_title', 'q_body', 'accepted_answer_id', 'q_creation_date',
       'q_tags', 'q_view_count', 'q_score_tier'],
      dtype='object')

In [68]:
# Determine length of title
posts_questions_df['q_title_char_count']= posts_questions_df['q_title'].str.len() #character count
posts_questions_df['q_title_word_count']= posts_questions_df['q_title'].str.split().str.len() #word count
posts_questions_df.head()

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_char_count,q_title_word_count
0,67341742,Is there any type of way to call a function in...,<p>sorry in advance for my lack of technical l...,67341801,2021-05-01 00:03:03.923000+00:00,javascript|html|node.js|obs,34,Zero Score (0),95,20
1,67341817,Produce multiple models for CVC4 SMT queries,<p>Can I get multiple models for a query like ...,67341857,2021-05-01 00:17:39.380000+00:00,z3|smt|formal-methods|satisfiability|cvc4,31,Zero Score (0),44,7
2,67341895,Why does shuffling array 'A' also shuffle arra...,<p>I'm trying to make a shuffled copy of an ar...,67341911,2021-05-01 00:32:18.187000+00:00,python|numpy,49,Positive Score (>0),97,17
3,67341936,Is there a simpler way to validate user input ...,<p>I am currently learning python and I was wo...,67341961,2021-05-01 00:41:17.673000+00:00,python,43,Positive Score (>0),86,15
4,67341921,How to Install vue3 in Laravel 8 without vue-s...,<p>I have install Laravel 8 fresh project. And...,67341974,2021-05-01 00:38:52.743000+00:00,laravel-8|vuejs3,18,Zero Score (0),114,17


In [69]:
# Bin by title length (char count)
q_title_explore_char = posts_questions_df.groupby('q_title_char_count').count()
q_title_explore_char

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_word_count
q_title_char_count,,,,,,,,,
15,25,25,25,25,25,25,25,25,25
16,22,22,22,22,22,22,22,22,22
17,39,39,39,39,39,39,39,39,39
18,44,44,44,44,44,44,44,44,44
19,51,51,51,51,51,51,51,51,51
...,...,...,...,...,...,...,...,...,...
146,17,17,17,17,17,17,17,17,17
147,25,25,25,25,25,25,25,25,25
148,27,27,27,27,27,27,27,27,27


In [70]:
# Bin title length by char count

# Establish the bins.
q_char_count_bins = [0, 50, 100, 150]
q_char_count_group_names = ["Short (0 - 50)", "Medium (50-100)", "Long (100-150)"]

# Categorize char length based on the bins.
posts_questions_df['q_title_char_count_bin'] = pd.cut(posts_questions_df['q_title_char_count'], q_char_count_bins, labels=q_char_count_group_names)

# Bin by char length
q_char_count_explore = posts_questions_df.groupby('q_title_char_count_bin').count()
q_char_count_explore

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_char_count,q_title_word_count
q_title_char_count_bin,,,,,,,,,,
Short (0 - 50),18552,18552,18552,18552,18552,18552,18552,18552,18552,18552
Medium (50-100),26079,26079,26079,26079,26079,26079,26079,26079,26079,26079
Long (100-150),2426,2426,2426,2426,2426,2426,2426,2426,2426,2426


In [71]:
# Bin by title length (word count)
q_title_explore_word = posts_questions_df.groupby('q_title_word_count').count()
q_title_explore_word

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_char_count,q_title_char_count_bin
q_title_word_count,,,,,,,,,,
1,1,1,1,1,1,1,1,1,1,1
2,75,75,75,75,75,75,75,75,75,75
3,573,573,573,573,573,573,573,573,573,573
4,1452,1452,1452,1452,1452,1452,1452,1452,1452,1452
5,2755,2755,2755,2755,2755,2755,2755,2755,2755,2755
6,4072,4072,4072,4072,4072,4072,4072,4072,4072,4072
7,4857,4857,4857,4857,4857,4857,4857,4857,4857,4857
8,5422,5422,5422,5422,5422,5422,5422,5422,5422,5422
9,5283,5283,5283,5283,5283,5283,5283,5283,5283,5283


In [72]:
# Bin title length by word count

# Establish the bins.
q_title_word_count_bins = [0, 10, 20, 30, 40]
q_title_word_count_group_names = ["Short (0 - 10)", "Medium (10-20)", "Long (20-30)", "XL (30+)"]

# Categorize char length based on the bins.
posts_questions_df['q_title_word_count_bin'] = pd.cut(posts_questions_df['q_title_word_count'], q_title_word_count_bins, labels=q_title_word_count_group_names)

# Bin by char length
q_word_count_explore = posts_questions_df.groupby('q_title_word_count_bin').count()
q_word_count_explore

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_char_count,q_title_word_count,q_title_char_count_bin
q_title_word_count_bin,,,,,,,,,,,
Short (0 - 10),29266,29266,29266,29266,29266,29266,29266,29266,29266,29266,29266
Medium (10-20),16900,16900,16900,16900,16900,16900,16900,16900,16900,16900,16900
Long (20-30),877,877,877,877,877,877,877,877,877,877,877
XL (30+),14,14,14,14,14,14,14,14,14,14,14


In [73]:
# Clean up posts_questions_df
posts_questions_df.columns

Index(['q_id', 'q_title', 'q_body', 'accepted_answer_id', 'q_creation_date',
       'q_tags', 'q_view_count', 'q_score_tier', 'q_title_char_count',
       'q_title_word_count', 'q_title_char_count_bin',
       'q_title_word_count_bin'],
      dtype='object')

In [74]:
# Drop q_title_char_count and q_title_word_count columns because data transformed into bins
posts_questions_df = posts_questions_df.drop(columns='q_title_char_count')
posts_questions_df = posts_questions_df.drop(columns='q_title_word_count')
posts_questions_df.columns

Index(['q_id', 'q_title', 'q_body', 'accepted_answer_id', 'q_creation_date',
       'q_tags', 'q_view_count', 'q_score_tier', 'q_title_char_count_bin',
       'q_title_word_count_bin'],
      dtype='object')

In [84]:
# Explore view count
q_views_explore_raw = posts_questions_df.groupby('q_view_count').count()
q_views_explore_raw

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_score_tier,q_title_char_count_bin,q_title_word_count_bin
q_view_count,,,,,,,,,
5,2,2,2,2,2,2,2,2,2
6,7,7,7,7,7,7,7,7,7
7,10,10,10,10,10,10,10,10,10
8,28,28,28,28,28,28,28,28,28
9,56,56,56,56,56,56,56,56,56
...,...,...,...,...,...,...,...,...,...
8221,1,1,1,1,1,1,1,1,1
8939,1,1,1,1,1,1,1,1,1
9996,1,1,1,1,1,1,1,1,1


In [87]:
q_views_explore_raw.sort_values(by=['q_id']).tail(20) # Majority views around mid 20s to 30s

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_score_tier,q_title_char_count_bin,q_title_word_count_bin
q_view_count,,,,,,,,,
38,1057,1057,1057,1057,1057,1057,1057,1057,1057
37,1078,1078,1078,1078,1078,1078,1078,1078,1078
19,1085,1085,1085,1085,1085,1085,1085,1085,1085
20,1163,1163,1163,1163,1163,1163,1163,1163,1163
36,1171,1171,1171,1171,1171,1171,1171,1171,1171
35,1217,1217,1217,1217,1217,1217,1217,1217,1217
34,1255,1255,1255,1255,1255,1255,1255,1255,1255
21,1260,1260,1260,1260,1260,1260,1260,1260,1260
33,1275,1275,1275,1275,1275,1275,1275,1275,1275


In [88]:
# Bin view count for more meaningful analysis
# Establish the bins.
q_view_count_bins = [0,  50, 500, 1000, 5000, 16000]
q_view_count_group_names = ["<50", "50-500", "500-1000", "1000-5000", "5000-16000"]

# Categorize score based on the bins.
posts_questions_df['q_view_count_bin'] = pd.cut(posts_questions_df['q_view_count'], q_view_count_bins, labels=q_view_count_group_names)

In [89]:
# Explore view count
q_views_explore = posts_questions_df.groupby('q_view_count_bin').count()
q_views_explore

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_char_count_bin,q_title_word_count_bin
q_view_count_bin,,,,,,,,,,
<50,39372,39372,39372,39372,39372,39372,39372,39372,39372,39372
50-500,7618,7618,7618,7618,7618,7618,7618,7618,7618,7618
500-1000,36,36,36,36,36,36,36,36,36,36
1000-5000,25,25,25,25,25,25,25,25,25,25
5000-16000,6,6,6,6,6,6,6,6,6,6


In [94]:
# Readjust bins for more distributed data - add meaningfulness to ML model inputs

# Establish the bins.
q_view_count_bins = [0, 10, 20, 30, 40, 50, 16000]
q_view_count_group_names = ["<10", "10-20", "20-30","30-40", "40-50", "50-16000"]

# Categorize score based on the bins.
posts_questions_df['q_view_count_bin'] = pd.cut(posts_questions_df['q_view_count'], q_view_count_bins, labels=q_view_count_group_names)
# Explore view count
q_views_explore = posts_questions_df.groupby('q_view_count_bin').count()
q_views_explore

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_char_count_bin,q_title_word_count_bin
q_view_count_bin,,,,,,,,,,
<10,215,215,215,215,215,215,215,215,215,215
10-20,6462,6462,6462,6462,6462,6462,6462,6462,6462,6462
20-30,14153,14153,14153,14153,14153,14153,14153,14153,14153,14153
30-40,11636,11636,11636,11636,11636,11636,11636,11636,11636,11636
40-50,6906,6906,6906,6906,6906,6906,6906,6906,6906,6906
50-16000,7685,7685,7685,7685,7685,7685,7685,7685,7685,7685


In [95]:
# Determine word count of body
posts_questions_df['q_body_word_count']= posts_questions_df['q_body'].str.split().str.len() #word count
# Group by body word count
q_body_explore_raw = posts_questions_df.groupby('q_body_word_count').count()
q_body_explore_raw

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_char_count_bin,q_title_word_count_bin,q_view_count_bin
q_body_word_count,,,,,,,,,,,
6,1,1,1,1,1,1,1,1,1,1,1
7,1,1,1,1,1,1,1,1,1,1,1
8,2,2,2,2,2,2,2,2,2,2,2
9,2,2,2,2,2,2,2,2,2,2,2
10,3,3,3,3,3,3,3,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...
3393,1,1,1,1,1,1,1,1,1,1,1
3472,1,1,1,1,1,1,1,1,1,1,1
4030,1,1,1,1,1,1,1,1,1,1,1


In [96]:
q_body_explore_raw.sort_values(by=['q_id']).tail(20) # Majority views around mid 20s to 30s

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_char_count_bin,q_title_word_count_bin,q_view_count_bin
q_body_word_count,,,,,,,,,,,
80,239,239,239,239,239,239,239,239,239,239,239
102,240,240,240,240,240,240,240,240,240,240,240
83,242,242,242,242,242,242,242,242,242,242,242
88,242,242,242,242,242,242,242,242,242,242,242
111,244,244,244,244,244,244,244,244,244,244,244
89,244,244,244,244,244,244,244,244,244,244,244
106,246,246,246,246,246,246,246,246,246,246,246
100,247,247,247,247,247,247,247,247,247,247,247
101,248,248,248,248,248,248,248,248,248,248,248


In [97]:
# Bin body word count for more meaningful analysis
# Establish the bins.
q_body_len_bins = [0,  100, 500, 10000]
q_body_len_group_names = ["<100", "100-500", "500-1000", "1000-5000", "5000-10000"]

# Categorize body length based on the bins.
posts_questions_df['q_body_len_bin'] = pd.cut(posts_questions_df['q_body_word_count'], q_body_len_bins, labels=q_body_len_group_names)

# Explore body length
q_body_len_explore = posts_questions_df.groupby('q_body_len_bin').count()
q_body_len_explore

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_char_count_bin,q_title_word_count_bin,q_view_count_bin,q_body_word_count
q_body_len_bin,,,,,,,,,,,,
<100,14766,14766,14766,14766,14766,14766,14766,14766,14766,14766,14766,14766
100-500,30381,30381,30381,30381,30381,30381,30381,30381,30381,30381,30381,30381
500-1000,1595,1595,1595,1595,1595,1595,1595,1595,1595,1595,1595,1595
1000-5000,313,313,313,313,313,313,313,313,313,313,313,313
5000-10000,2,2,2,2,2,2,2,2,2,2,2,2


In [98]:
# Readjust bins for more distributed data - add meaningfulness to ML model inputs

# Establish the bins.
q_body_len_bins = [0, 50, 100, 250, 500, 10000]
q_body_len_group_names = ["<50", "50-100", "100-250", "250-500", "500-10000"]

# Categorize body length based on the bins.
posts_questions_df['q_body_len_bin'] = pd.cut(posts_questions_df['q_body_word_count'], q_body_len_bins, labels=q_body_len_group_names)

# Explore body length
q_body_len_explore = posts_questions_df.groupby('q_body_len_bin').count()
q_body_len_explore

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_char_count_bin,q_title_word_count_bin,q_view_count_bin,q_body_word_count
q_body_len_bin,,,,,,,,,,,,
<50,3389,3389,3389,3389,3389,3389,3389,3389,3389,3389,3389,3389
50-100,11377,11377,11377,11377,11377,11377,11377,11377,11377,11377,11377,11377
100-250,22844,22844,22844,22844,22844,22844,22844,22844,22844,22844,22844,22844
250-500,7537,7537,7537,7537,7537,7537,7537,7537,7537,7537,7537,7537
500-10000,1910,1910,1910,1910,1910,1910,1910,1910,1910,1910,1910,1910


In [99]:
# Create list with all values within q_body
q_body = posts_questions_df['q_body']
q_body

0        <p>sorry in advance for my lack of technical l...
1        <p>Can I get multiple models for a query like ...
2        <p>I'm trying to make a shuffled copy of an ar...
3        <p>I am currently learning python and I was wo...
4        <p>I have install Laravel 8 fresh project. And...
                               ...                        
47052    <p>I want to make a program that randomly simu...
47053    <p>This is a very simple protgresql haskell ex...
47054    <p>=IMPORTXML(&quot;https://www.klsescreener.c...
47055    <p>i am struggling with RegEx, i want to targe...
47056    <p>I am using the snippet from <a href="https:...
Name: q_body, Length: 47057, dtype: object

In [ ]:
# Find all unique HTML tags in q_body list
    # Use re library instead of hardcoding things
    # Use functions because it's a repetitive process

# Count number of each unique tag for every row in q_body list
    # For each row, how many <p>, how many <img tags>, etc
    # Store as separate column in q_body
    
# Group by tags


In [ ]:
# Create function to count tags
    # Create list from tags column
    # Count number of "|" + 1

In [ ]:
# Explore the Votes table
    # Filterby date
    # Filterby Question post
    # Group by
# Expand our query beyond May 2021, go back 12 months

# Create our own tags table and categorize and encode it

In [28]:
# Pull only relevant columns to ML model
# Sample data limited to June 2021, will scale up for next steps
query_job = client.query(
        """
        SELECT
        id, accepted_answer_id, creation_date, tags
        FROM `bigquery-public-data.stackoverflow.posts_questions`
        WHERE accepted_answer_id is not null AND creation_date > '2021-05-01'
        ORDER BY accepted_answer_id"""
    )

results = query_job.result()  # Waits for job to complete.
    # [END bigquery_simple_app_query]

    # [START bigquery_simple_app_print]
# for row in results:
#     print(row)
results_list = [row for row in results]

In [29]:
cols = ['id', 'accepted_answer_id', 'question_creation_date', 'tags']

In [30]:
post_questions = pd.DataFrame.from_records(results_list, columns=cols)
post_questions

,id,accepted_answer_id,question_creation_date,tags
0,67341742,67341801,2021-05-01 00:03:03.923000+00:00,javascript|html|node.js|obs
1,67341817,67341857,2021-05-01 00:17:39.380000+00:00,z3|smt|formal-methods|satisfiability|cvc4
2,67341895,67341911,2021-05-01 00:32:18.187000+00:00,python|numpy
3,67341936,67341961,2021-05-01 00:41:17.673000+00:00,python
4,67341921,67341974,2021-05-01 00:38:52.743000+00:00,laravel-8|vuejs3
...,...,...,...,...
47052,67531286,67757704,2021-05-14 08:28:20.750000+00:00,python|random
47053,67757579,67757716,2021-05-30 04:02:47.097000+00:00,postgresql|haskell
47054,67757723,67757776,2021-05-30 04:38:50.357000+00:00,html|google-sheets
47055,67757792,67757830,2021-05-30 04:50:54.750000+00:00,javascript|regex


In [31]:
query_job = client.query(
        """
        SELECT
        *
        FROM `bigquery-public-data.stackoverflow.posts_answers`
        WHERE creation_date > '2021-01-01'
        ORDER BY id
        LIMIT 10"""
    )

results = query_job.result()  # Waits for job to complete.
    # [END bigquery_simple_app_query]

    # [START bigquery_simple_app_print]
# for row in results:
#     print(row)
results_list = [row for row in results]

In [32]:
cols = [field.name for field in results.schema]

In [33]:
post_answers = pd.DataFrame.from_records(results_list, columns=cols)
post_answers

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,65526388,None,<p>Here is one solution. You can first do a gr...,None,None,0,None,2021-01-01 00:00:01.653000+00:00,None,2021-01-01 00:00:01.653000+00:00,NaT,None,NaN,None,4520420,65525964,2,2,None,None
1,65526389,None,<p>Echoing from the individual threads can be ...,None,None,0,None,2021-01-01 00:00:11.187000+00:00,None,2021-01-01 00:00:11.187000+00:00,NaT,None,NaN,None,342497,54217345,2,0,None,None
2,65526391,None,"<p>I have found similar code here (<a href=""ht...",None,None,5,None,2021-01-01 00:00:41.780000+00:00,None,2021-01-01 00:00:41.780000+00:00,NaT,None,NaN,None,13436156,65526354,2,0,None,None
3,65526393,None,"<p>Encountered the same error, just wanna add ...",None,None,0,None,2021-01-01 00:01:25.723000+00:00,None,2021-01-01 00:01:25.723000+00:00,NaT,None,NaN,None,12552972,48184969,2,0,None,None
4,65526394,None,<p>It's difficult to have all three types of v...,None,None,6,None,2021-01-01 00:01:33.950000+00:00,None,2021-01-01 00:01:33.950000+00:00,NaT,None,NaN,None,12602208,65525938,2,0,None,None
5,65526395,None,<p>to be able to change other user passwords y...,None,None,0,None,2021-01-01 00:01:32.977000+00:00,None,2021-01-01 00:01:32.977000+00:00,NaT,None,NaN,None,5562372,65523941,2,0,None,None
6,65526398,None,<blockquote>\n<p>Is it safe (and possible?) to...,None,None,2,None,2021-01-01 00:01:52.123000+00:00,None,2021-01-01 00:09:28.470000+00:00,2021-01-01 00:09:28.470000+00:00,None,14660.0,None,14660,65526234,2,1,None,None
7,65526403,None,<pre><code>def shuffle(deck):\n split_deck ...,None,None,0,None,2021-01-01 00:02:34.570000+00:00,None,2021-01-01 00:02:34.570000+00:00,NaT,None,NaN,None,4718350,65526315,2,1,None,None
8,65526404,None,"<p>FYI, here's a simple app that will do what ...",None,None,1,None,2021-01-01 00:02:40.090000+00:00,None,2021-01-01 00:09:09.673000+00:00,2021-01-01 00:09:09.673000+00:00,None,816620.0,None,816620,65514069,2,0,None,None
9,65526405,None,<p>First you'll want to check if there is no r...,None,None,0,None,2021-01-01 00:02:40.743000+00:00,None,2021-01-01 00:22:23.763000+00:00,2021-01-01 00:22:23.763000+00:00,None,14560865.0,None,14560865,65526331,2,2,None,None


In [34]:
# Pull only relevant columns to ML model
query_job = client.query(
        """
        SELECT
        id, creation_date
        FROM `bigquery-public-data.stackoverflow.posts_answers`
        WHERE creation_date > '2021-05-01'
        ORDER BY id"""
    )

results = query_job.result()  # Waits for job to complete.
    # [END bigquery_simple_app_query]

    # [START bigquery_simple_app_print]
# for row in results:
#     print(row)
results_list = [row for row in results]

In [35]:
cols = ['answer_id', 'answer_creation_date']

In [36]:
post_answers = pd.DataFrame.from_records(results_list, columns=cols)
post_answers

,answer_id,answer_creation_date
0,67341731,2021-05-01 00:00:43.057000+00:00
1,67341735,2021-05-01 00:02:14.853000+00:00
2,67341736,2021-05-01 00:02:18.417000+00:00
3,67341740,2021-05-01 00:02:59.713000+00:00
4,67341741,2021-05-01 00:03:03.637000+00:00
...,...,...
169650,67757977,2021-05-30 05:25:20.073000+00:00
169651,67757978,2021-05-30 05:25:23.510000+00:00
169652,67757979,2021-05-30 05:25:34.620000+00:00
169653,67757981,2021-05-30 05:25:46.913000+00:00


In [37]:
post_questions = post_questions.rename(columns={"id":"parent_id"})
post_questions

,parent_id,accepted_answer_id,question_creation_date,tags
0,67341742,67341801,2021-05-01 00:03:03.923000+00:00,javascript|html|node.js|obs
1,67341817,67341857,2021-05-01 00:17:39.380000+00:00,z3|smt|formal-methods|satisfiability|cvc4
2,67341895,67341911,2021-05-01 00:32:18.187000+00:00,python|numpy
3,67341936,67341961,2021-05-01 00:41:17.673000+00:00,python
4,67341921,67341974,2021-05-01 00:38:52.743000+00:00,laravel-8|vuejs3
...,...,...,...,...
47052,67531286,67757704,2021-05-14 08:28:20.750000+00:00,python|random
47053,67757579,67757716,2021-05-30 04:02:47.097000+00:00,postgresql|haskell
47054,67757723,67757776,2021-05-30 04:38:50.357000+00:00,html|google-sheets
47055,67757792,67757830,2021-05-30 04:50:54.750000+00:00,javascript|regex


In [38]:
merged_df = pd.merge(post_questions, post_answers, left_on=  ['accepted_answer_id'],
                   right_on= ['answer_id'], 
                   how = 'left')
merged_df

,parent_id,accepted_answer_id,question_creation_date,tags,answer_id,answer_creation_date
0,67341742,67341801,2021-05-01 00:03:03.923000+00:00,javascript|html|node.js|obs,67341801,2021-05-01 00:14:38.340000+00:00
1,67341817,67341857,2021-05-01 00:17:39.380000+00:00,z3|smt|formal-methods|satisfiability|cvc4,67341857,2021-05-01 00:26:39.053000+00:00
2,67341895,67341911,2021-05-01 00:32:18.187000+00:00,python|numpy,67341911,2021-05-01 00:36:54.910000+00:00
3,67341936,67341961,2021-05-01 00:41:17.673000+00:00,python,67341961,2021-05-01 00:46:37.893000+00:00
4,67341921,67341974,2021-05-01 00:38:52.743000+00:00,laravel-8|vuejs3,67341974,2021-05-01 00:48:52.390000+00:00
...,...,...,...,...,...,...
47052,67531286,67757704,2021-05-14 08:28:20.750000+00:00,python|random,67757704,2021-05-30 04:35:06.557000+00:00
47053,67757579,67757716,2021-05-30 04:02:47.097000+00:00,postgresql|haskell,67757716,2021-05-30 04:37:58.840000+00:00
47054,67757723,67757776,2021-05-30 04:38:50.357000+00:00,html|google-sheets,67757776,2021-05-30 04:48:57.807000+00:00
47055,67757792,67757830,2021-05-30 04:50:54.750000+00:00,javascript|regex,67757830,2021-05-30 04:57:39.123000+00:00


In [39]:
# Compare accepted_answer_id and answer_id to verify identical
merged_df['accepted_answer_id'].equals(merged_df['answer_id'])

True

In [40]:
# Drop "answer_id" column because redundant
merged_df = merged_df.drop(columns=['answer_id'])
merged_df

,parent_id,accepted_answer_id,question_creation_date,tags,answer_creation_date
0,67341742,67341801,2021-05-01 00:03:03.923000+00:00,javascript|html|node.js|obs,2021-05-01 00:14:38.340000+00:00
1,67341817,67341857,2021-05-01 00:17:39.380000+00:00,z3|smt|formal-methods|satisfiability|cvc4,2021-05-01 00:26:39.053000+00:00
2,67341895,67341911,2021-05-01 00:32:18.187000+00:00,python|numpy,2021-05-01 00:36:54.910000+00:00
3,67341936,67341961,2021-05-01 00:41:17.673000+00:00,python,2021-05-01 00:46:37.893000+00:00
4,67341921,67341974,2021-05-01 00:38:52.743000+00:00,laravel-8|vuejs3,2021-05-01 00:48:52.390000+00:00
...,...,...,...,...,...
47052,67531286,67757704,2021-05-14 08:28:20.750000+00:00,python|random,2021-05-30 04:35:06.557000+00:00
47053,67757579,67757716,2021-05-30 04:02:47.097000+00:00,postgresql|haskell,2021-05-30 04:37:58.840000+00:00
47054,67757723,67757776,2021-05-30 04:38:50.357000+00:00,html|google-sheets,2021-05-30 04:48:57.807000+00:00
47055,67757792,67757830,2021-05-30 04:50:54.750000+00:00,javascript|regex,2021-05-30 04:57:39.123000+00:00


In [41]:
# Add column for day of question_creation_date [question_day]
merged_df['question_day'] = merged_df['question_creation_date'].dt.day_name()
merged_df

,parent_id,accepted_answer_id,question_creation_date,tags,answer_creation_date,question_day
0,67341742,67341801,2021-05-01 00:03:03.923000+00:00,javascript|html|node.js|obs,2021-05-01 00:14:38.340000+00:00,Saturday
1,67341817,67341857,2021-05-01 00:17:39.380000+00:00,z3|smt|formal-methods|satisfiability|cvc4,2021-05-01 00:26:39.053000+00:00,Saturday
2,67341895,67341911,2021-05-01 00:32:18.187000+00:00,python|numpy,2021-05-01 00:36:54.910000+00:00,Saturday
3,67341936,67341961,2021-05-01 00:41:17.673000+00:00,python,2021-05-01 00:46:37.893000+00:00,Saturday
4,67341921,67341974,2021-05-01 00:38:52.743000+00:00,laravel-8|vuejs3,2021-05-01 00:48:52.390000+00:00,Saturday
...,...,...,...,...,...,...
47052,67531286,67757704,2021-05-14 08:28:20.750000+00:00,python|random,2021-05-30 04:35:06.557000+00:00,Friday
47053,67757579,67757716,2021-05-30 04:02:47.097000+00:00,postgresql|haskell,2021-05-30 04:37:58.840000+00:00,Sunday
47054,67757723,67757776,2021-05-30 04:38:50.357000+00:00,html|google-sheets,2021-05-30 04:48:57.807000+00:00,Sunday
47055,67757792,67757830,2021-05-30 04:50:54.750000+00:00,javascript|regex,2021-05-30 04:57:39.123000+00:00,Sunday


In [42]:
# Add column for hour value of question_creation_date [question_time]
merged_df['question_hour'] = merged_df['question_creation_date'].dt.hour
merged_df['question_hour_min'] = merged_df['question_creation_date'].dt.strftime('%H:%M')

merged_df

,parent_id,accepted_answer_id,question_creation_date,tags,answer_creation_date,question_day,question_hour,question_hour_min
0,67341742,67341801,2021-05-01 00:03:03.923000+00:00,javascript|html|node.js|obs,2021-05-01 00:14:38.340000+00:00,Saturday,0,00:03
1,67341817,67341857,2021-05-01 00:17:39.380000+00:00,z3|smt|formal-methods|satisfiability|cvc4,2021-05-01 00:26:39.053000+00:00,Saturday,0,00:17
2,67341895,67341911,2021-05-01 00:32:18.187000+00:00,python|numpy,2021-05-01 00:36:54.910000+00:00,Saturday,0,00:32
3,67341936,67341961,2021-05-01 00:41:17.673000+00:00,python,2021-05-01 00:46:37.893000+00:00,Saturday,0,00:41
4,67341921,67341974,2021-05-01 00:38:52.743000+00:00,laravel-8|vuejs3,2021-05-01 00:48:52.390000+00:00,Saturday,0,00:38
...,...,...,...,...,...,...,...,...
47052,67531286,67757704,2021-05-14 08:28:20.750000+00:00,python|random,2021-05-30 04:35:06.557000+00:00,Friday,8,08:28
47053,67757579,67757716,2021-05-30 04:02:47.097000+00:00,postgresql|haskell,2021-05-30 04:37:58.840000+00:00,Sunday,4,04:02
47054,67757723,67757776,2021-05-30 04:38:50.357000+00:00,html|google-sheets,2021-05-30 04:48:57.807000+00:00,Sunday,4,04:38
47055,67757792,67757830,2021-05-30 04:50:54.750000+00:00,javascript|regex,2021-05-30 04:57:39.123000+00:00,Sunday,4,04:50


In [43]:
# Add column for duration till accepted answer (answer_creation_date - question_creation_date)
merged_df['accepted_answer_duration'] = (merged_df['answer_creation_date']-merged_df['question_creation_date'])/np.timedelta64(1,'h')
merged_df = merged_df.sort_values(by='accepted_answer_duration', ascending=False)
merged_df

,parent_id,accepted_answer_id,question_creation_date,tags,answer_creation_date,question_day,question_hour,question_hour_min,accepted_answer_duration
46975,67342099,67756344,2021-05-01 01:19:14.210000+00:00,ios|swift|uibezierpath|rounded-corners|spacing,2021-05-29 22:57:23.117000+00:00,Saturday,1,01:19,693.635808
45777,67349217,67741551,2021-05-01 17:43:24.997000+00:00,javascript|css,2021-05-28 15:18:48.583000+00:00,Saturday,17,17:43,645.589885
46098,67352708,67744939,2021-05-02 03:20:42.240000+00:00,python|django,2021-05-28 20:10:38.163000+00:00,Sunday,3,03:20,640.832201
41532,67349112,67699250,2021-05-01 17:34:03.697000+00:00,django|class|django-rest-framework,2021-05-26 05:57:48.013000+00:00,Saturday,17,17:34,588.395643
46603,67399034,67750961,2021-05-05 09:56:23.113000+00:00,java|drag-and-drop|awt|netbeans-platform,2021-05-29 11:44:36.820000+00:00,Wednesday,9,09:56,577.803807
...,...,...,...,...,...,...,...,...,...
25782,67563037,67563038,2021-05-17 01:35:04.927000+00:00,android|android-studio|android-layout|keyboard...,2021-05-17 01:35:04.927000+00:00,Monday,1,01:35,0.000000
16408,67485503,67485504,2021-05-11 11:05:40.837000+00:00,kubernetes|apache-kafka|apache-zookeeper|yahoo...,2021-05-11 11:05:40.837000+00:00,Tuesday,11,11:05,0.000000
37859,67666770,67666771,2021-05-24 04:42:53.953000+00:00,mpdf,2021-05-24 04:42:53.953000+00:00,Monday,4,04:42,0.000000
25652,67561998,67561999,2021-05-16 22:14:34.137000+00:00,c#|formatting|t4,2021-05-16 22:14:34.137000+00:00,Sunday,22,22:14,0.000000


In [44]:
# Additional method to verify number of rows with zero response time (what's the possible explanation?)
zero_time = merged_df.groupby('accepted_answer_duration').count()
zero_time 

,parent_id,accepted_answer_id,question_creation_date,tags,answer_creation_date,question_day,question_hour,question_hour_min
accepted_answer_duration,,,,,,,,
0.000000,172,172,172,172,172,172,172,172
0.004931,1,1,1,1,1,1,1,1
0.005020,1,1,1,1,1,1,1,1
0.006600,1,1,1,1,1,1,1,1
0.007744,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...
577.803807,1,1,1,1,1,1,1,1
588.395643,1,1,1,1,1,1,1,1
640.832201,1,1,1,1,1,1,1,1


In [45]:
# drop data rows where duration is 0
indexNames = merged_df[ merged_df['accepted_answer_duration'] == 0.000000 ].index
merged_df.drop(indexNames, inplace=True)
merged_df

,parent_id,accepted_answer_id,question_creation_date,tags,answer_creation_date,question_day,question_hour,question_hour_min,accepted_answer_duration
46975,67342099,67756344,2021-05-01 01:19:14.210000+00:00,ios|swift|uibezierpath|rounded-corners|spacing,2021-05-29 22:57:23.117000+00:00,Saturday,1,01:19,693.635808
45777,67349217,67741551,2021-05-01 17:43:24.997000+00:00,javascript|css,2021-05-28 15:18:48.583000+00:00,Saturday,17,17:43,645.589885
46098,67352708,67744939,2021-05-02 03:20:42.240000+00:00,python|django,2021-05-28 20:10:38.163000+00:00,Sunday,3,03:20,640.832201
41532,67349112,67699250,2021-05-01 17:34:03.697000+00:00,django|class|django-rest-framework,2021-05-26 05:57:48.013000+00:00,Saturday,17,17:34,588.395643
46603,67399034,67750961,2021-05-05 09:56:23.113000+00:00,java|drag-and-drop|awt|netbeans-platform,2021-05-29 11:44:36.820000+00:00,Wednesday,9,09:56,577.803807
...,...,...,...,...,...,...,...,...,...
12265,67450739,67450745,2021-05-08 18:12:53.640000+00:00,r|dataframe|ggplot2,2021-05-08 18:13:25.640000+00:00,Saturday,18,18:12,0.008889
12316,67451178,67451182,2021-05-08 19:03:44.257000+00:00,r|string,2021-05-08 19:04:12.137000+00:00,Saturday,19,19:03,0.007744
19631,67512565,67512567,2021-05-13 00:35:23.640000+00:00,sql|sql-server|tsql,2021-05-13 00:35:47.400000+00:00,Thursday,0,00:35,0.006600
8154,67415161,67415165,2021-05-06 09:13:30.220000+00:00,spring-boot|mybatis|lombok,2021-05-06 09:13:48.293000+00:00,Thursday,9,09:13,0.005020


In [46]:
# create a subset of merged_df for ML model
#response_duration_ML_df = merged_df['accepted_answer_id', 'question_day', 'question_hour', 'question_hour_min', 'accepted_answer_duration']
#response_duration_ML_df

practice_ML_df = merged_df[['accepted_answer_id', 'question_creation_date', 'question_day', 'question_hour', 'question_hour_min', 'accepted_answer_duration']]
practice_ML_df

,accepted_answer_id,question_creation_date,question_day,question_hour,question_hour_min,accepted_answer_duration
46975,67756344,2021-05-01 01:19:14.210000+00:00,Saturday,1,01:19,693.635808
45777,67741551,2021-05-01 17:43:24.997000+00:00,Saturday,17,17:43,645.589885
46098,67744939,2021-05-02 03:20:42.240000+00:00,Sunday,3,03:20,640.832201
41532,67699250,2021-05-01 17:34:03.697000+00:00,Saturday,17,17:34,588.395643
46603,67750961,2021-05-05 09:56:23.113000+00:00,Wednesday,9,09:56,577.803807
...,...,...,...,...,...,...
12265,67450745,2021-05-08 18:12:53.640000+00:00,Saturday,18,18:12,0.008889
12316,67451182,2021-05-08 19:03:44.257000+00:00,Saturday,19,19:03,0.007744
19631,67512567,2021-05-13 00:35:23.640000+00:00,Thursday,0,00:35,0.006600
8154,67415165,2021-05-06 09:13:30.220000+00:00,Thursday,9,09:13,0.005020


In [47]:
# Reorder columns

In [48]:
query_job = client.query(
        """
        SELECT
        *
        FROM `bigquery-public-data.stackoverflow.votes`
        LIMIT 10"""
    )

results = query_job.result()  # Waits for job to complete.
    # [END bigquery_simple_app_query]

    # [START bigquery_simple_app_print]
# for row in results:
#     print(row)
results_list = [row for row in results]
cols = [field.name for field in results.schema]
votes = pd.DataFrame.from_records(results_list, columns=cols)
votes

,id,creation_date,post_id,vote_type_id
0,14472571,2011-04-25 00:00:00+00:00,5774048,2
1,14473282,2011-04-25 00:00:00+00:00,5774283,2
2,14478256,2011-04-25 00:00:00+00:00,3086484,2
3,14479599,2011-04-25 00:00:00+00:00,151005,5
4,14479762,2011-04-25 00:00:00+00:00,5663770,10
5,14481320,2011-04-25 00:00:00+00:00,267761,5
6,14482412,2011-04-25 00:00:00+00:00,5733809,2
7,14483023,2011-04-25 00:00:00+00:00,5778782,2
8,14483233,2011-04-25 00:00:00+00:00,5778696,2
9,14483799,2011-04-25 00:00:00+00:00,5774515,1


In [49]:
query_job = client.query(
        """
        SELECT
        *
        FROM `bigquery-public-data.stackoverflow.votes`
        LIMIT 10"""
    )

results = query_job.result()  # Waits for job to complete.
    # [END bigquery_simple_app_query]

    # [START bigquery_simple_app_print]
# for row in results:
#     print(row)
results_list = [row for row in results]
cols = [field.name for field in results.schema]
votes = pd.DataFrame.from_records(results_list, columns=cols)
votes

,id,creation_date,post_id,vote_type_id
0,14472571,2011-04-25 00:00:00+00:00,5774048,2
1,14473282,2011-04-25 00:00:00+00:00,5774283,2
2,14478256,2011-04-25 00:00:00+00:00,3086484,2
3,14479599,2011-04-25 00:00:00+00:00,151005,5
4,14479762,2011-04-25 00:00:00+00:00,5663770,10
5,14481320,2011-04-25 00:00:00+00:00,267761,5
6,14482412,2011-04-25 00:00:00+00:00,5733809,2
7,14483023,2011-04-25 00:00:00+00:00,5778782,2
8,14483233,2011-04-25 00:00:00+00:00,5778696,2
9,14483799,2011-04-25 00:00:00+00:00,5774515,1
